# Final Portfolio Project 2026
## 5CS037 - Concepts and Technologies of AI

**Student Name:** Biplov Maharjan  
**Student ID:** 2462258  

---

**Note:** This notebook implements Machine Learning algorithms **from scratch** using `numpy` to demonstrate a deep understanding of the underlying mathematical principles. `scikit-learn` is used primarily for data utilities (splitting, scaling) and performance metrics.

## Table of Contents
1. [Setup and Initialization](#setup)
2. [Models from Scratch](#models)
3. [Classification Task](#classification)
4. [Regression Task](#regression)


## 1. Setup and Initialization <a id='setup'></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Minimal Sklearn usage for utilities
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Set plot style
sns.set(style="whitegrid")
plt.rcParams['figure.figsize'] = (12, 8)

print("Libraries imported successfully.")

## 2. Models from Scratch <a id='models'></a>

In this section, I implemented the core logic for Neural Networks, Logistic Regression, Linear Regression, and KNN using NumPy matrix operations.

In [ ]:
class NeuralNetwork:
    def __init__(self, layers_structure, learning_rate=0.01, activation='relu', task='classification'):
        self.layers_structure = layers_structure  # List of neurons per layer e.g., [input_dim, 64, 32, 1]
        self.learning_rate = learning_rate
        self.hidden_activation = activation
        self.task = task  # 'classification' or 'regression'
        self.params = {}
        self.init_weights()
        
    def init_weights(self):
        np.random.seed(42)
        for i in range(1, len(self.layers_structure)):
            self.params[f'W{i}'] = np.random.randn(self.layers_structure[i-1], self.layers_structure[i]) * 0.01
            self.params[f'b{i}'] = np.zeros((1, self.layers_structure[i]))
            
    def sigmoid(self, Z):
        return 1 / (1 + np.exp(-Z))
    
    def relu(self, Z):
        return np.maximum(0, Z)
    
    def relu_deriv(self, Z):
        return Z > 0
    
    def forward(self, X):
        self.cache = {'A0': X}
        L = len(self.layers_structure) - 1
        
        for i in range(1, L + 1):
            W = self.params[f'W{i}']
            b = self.params[f'b{i}']
            A_prev = self.cache[f'A{i-1}']
            
            Z = np.dot(A_prev, W) + b
            self.cache[f'Z{i}'] = Z
            
            if i == L:  # Output Layer
                if self.task == 'classification':
                    A = self.sigmoid(Z)
                else:  # Regression (Linear)
                    A = Z
            else:  # Hidden Layer
                A = self.relu(Z) if self.hidden_activation == 'relu' else self.sigmoid(Z)
            
            self.cache[f'A{i}'] = A
            
        return self.cache[f'A{L}']
    
    def backward(self, Y):
        L = len(self.layers_structure) - 1
        m = Y.shape[0]
        gradients = {}
        
        # Initialize Backprop (Error at Output)
        A_final = self.cache[f'A{L}']
        
        if self.task == 'classification':
            dA = - (np.divide(Y, A_final + 1e-8) - np.divide(1 - Y, 1 - A_final + 1e-8))
            dZ = A_final - Y # Simplification for Sigmoid + CrossEntropy
        else:
            dZ = 2 * (A_final - Y) / m  # MSE Derivative
            
        gradients[f'dZ{L}'] = dZ
        gradients[f'dW{L}'] = np.dot(self.cache[f'A{L-1}'].T, dZ) / m
        gradients[f'db{L}'] = np.sum(dZ, axis=0, keepdims=True) / m
        
        # Backprop through Hidden Layers
        for i in range(L-1, 0, -1):
            dZ_next = gradients[f'dZ{i+1}']
            W_next = self.params[f'W{i+1}']
            
            dA = np.dot(dZ_next, W_next.T)
            Z = self.cache[f'Z{i}']
            
            if self.hidden_activation == 'relu':
                dZ = dA * self.relu_deriv(Z)
            else:
                s = self.sigmoid(Z)
                dZ = dA * s * (1 - s)
                
            gradients[f'dZ{i}'] = dZ
            gradients[f'dW{i}'] = np.dot(self.cache[f'A{i-1}'].T, dZ) / m
            gradients[f'db{i}'] = np.sum(dZ, axis=0, keepdims=True) / m
            
        return gradients
    
    def update_params(self, gradients):
        L = len(self.layers_structure) - 1
        for i in range(1, L + 1):
            self.params[f'W{i}'] -= self.learning_rate * gradients[f'dW{i}']
            self.params[f'b{i}'] -= self.learning_rate * gradients[f'db{i}']

    def fit(self, X, Y, epochs=1000, verbose=True):
        if Y.ndim == 1:
            Y = Y.reshape(-1, 1)
            
        loss_history = []
        
        for i in range(epochs):
            # Forward
            Y_hat = self.forward(X)
            
            # Loss Calculation
            if self.task == 'classification':
                loss = -np.mean(Y * np.log(Y_hat + 1e-8) + (1 - Y) * np.log(1 - Y_hat + 1e-8))
            else:
                loss = np.mean(np.square(Y - Y_hat))
            
            loss_history.append(loss)
            
            # Backward & Update
            grads = self.backward(Y)
            self.update_params(grads)
            
            if verbose and i % (epochs // 10) == 0:
                print(f"Epoch {i}: Loss {loss:.4f}")
                
        return loss_history
    
    def predict(self, X):
        Y_hat = self.forward(X)
        if self.task == 'classification':
            return (Y_hat > 0.5).astype(int)
        return Y_hat

In [ ]:
class LogisticRegressionScratch:
    def __init__(self, learning_rate=0.01, iterations=1000):
        self.lr = learning_rate
        self.iterations = iterations
        self.weights = None
        self.bias = None
        
    def sigmoid(self, z):
        return 1 / (1 + np.exp(-z))

    def fit(self, X, y):
        n_samples, n_features = X.shape
        self.weights = np.zeros(n_features)
        self.bias = 0

        # Gradient Descent
        for _ in range(self.iterations):
            model = np.dot(X, self.weights) + self.bias
            y_predicted = self.sigmoid(model)
            
            dw = (1 / n_samples) * np.dot(X.T, (y_predicted - y))
            db = (1 / n_samples) * np.sum(y_predicted - y)
            
            self.weights -= self.lr * dw
            self.bias -= self.lr * db

    def predict(self, X):
        linear_model = np.dot(X, self.weights) + self.bias
        y_predicted = self.sigmoid(linear_model)
        return [1 if i > 0.5 else 0 for i in y_predicted]

In [ ]:
class LinearRegressionScratch:
    def __init__(self, learning_rate=0.01, iterations=1000):
        self.lr = learning_rate
        self.iterations = iterations
        self.weights = None
        self.bias = None

    def fit(self, X, y):
        n_samples, n_features = X.shape
        self.weights = np.zeros(n_features)
        self.bias = 0
        
        for _ in range(self.iterations):
            y_predicted = np.dot(X, self.weights) + self.bias
            
            dw = (1 / n_samples) * np.dot(X.T, (y_predicted - y))
            db = (1 / n_samples) * np.sum(y_predicted - y)
            
            self.weights -= self.lr * dw
            self.bias -= self.lr * db

    def predict(self, X):
        return np.dot(X, self.weights) + self.bias

In [ ]:
class KNNScratch:
    def __init__(self, k=3, task='classification'):
        self.k = k
        self.task = task

    def fit(self, X, y):
        self.X_train = np.array(X)
        self.y_train = np.array(y)

    def predict(self, X):
        y_pred = [self._predict(x) for x in np.array(X)]
        return np.array(y_pred)

    def _predict(self, x):
        # Compute distances
        distances = [np.sqrt(np.sum((x_train - x) ** 2)) for x_train in self.X_train]
        # Get k nearest samples and their labels
        k_indices = np.argsort(distances)[:self.k]
        k_nearest_labels = [self.y_train[i] for i in k_indices]
        
        if self.task == 'classification':
            # Majority Vote
            most_common = max(set(k_nearest_labels), key=k_nearest_labels.count)
            return most_common
        else:
            # Mean
            return np.mean(k_nearest_labels)

## 3. Classification Task <a id='classification'></a>

We use the **Depression Student Dataset** for binary classification.
Target: `Depression` (0 or 1).

In [ ]:
# Load Data
class_data_path = r'classification/student_depression_dataset.csv'
try:
    df_class = pd.read_csv(class_data_path)
    # Preprocessing
    df_class = df_class.drop(columns=['id', 'City'], errors='ignore')
    
    # Encoding
    le = LabelEncoder()
    cat_cols = df_class.select_dtypes(include=['object']).columns
    for col in cat_cols:
        df_class[col] = le.fit_transform(df_class[col].astype(str))
    
    X = df_class.drop('Depression', axis=1).values # Convert to simple numpy array
    y = df_class['Depression'].values
    
    # Split and Scale
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    
    print("Classification Data Ready:", X_train.shape)
except Exception as e:
    print("Error loading data:", e)

In [ ]:
# 1. Train Neural Network (Scratch)
input_dim = X_train.shape[1]
nn_class = NeuralNetwork(layers_structure=[input_dim, 64, 32, 1], learning_rate=0.01, task='classification')

print("Training Custom Neural Network...")
loss_hist = nn_class.fit(X_train, y_train, epochs=2000, verbose=False)

plt.plot(loss_hist)
plt.title('NN Training Loss')
plt.show()

y_pred_nn = nn_class.predict(X_test)
print("Neural Network Evaluation:")
print(classification_report(y_test, y_pred_nn))

In [ ]:
# 2. Train Logistic Regression (Scratch)
log_reg = LogisticRegressionScratch(learning_rate=0.01, iterations=2000)
print("Training Custom Logistic Regression...")
log_reg.fit(X_train, y_train)

y_pred_log = log_reg.predict(X_test)
print("Logistic Regression Evaluation:")
print(classification_report(y_test, y_pred_log))

In [ ]:
# 3. Train KNN Classifier (Scratch)
# Using distinct subset to speed up KNN as it is computationally expensive
X_train_sub, _, y_train_sub, _ = train_test_split(X_train, y_train, train_size=2000, random_state=42)

knn = KNNScratch(k=5, task='classification')
print("Training Custom KNN...")
knn.fit(X_train_sub, y_train_sub)

y_pred_knn = knn.predict(X_test[:1000]) # Eval on subset for speed
print("KNN Evaluation (on subset):")
print(classification_report(y_test[:1000], y_pred_knn))

## 4. Regression Task <a id='regression'></a>

We use the **Avocado Dataset** for regression.
Target: `AveragePrice`.

In [ ]:
# Load Data
reg_data_path = r'regression/avocado.csv'
try:
    df_reg = pd.read_csv(reg_data_path)
    df_reg = df_reg.drop(columns=['Unnamed: 0', 'Date'], errors='ignore')
    
    le = LabelEncoder()
    for col in ['type', 'region']:
        df_reg[col] = le.fit_transform(df_reg[col])
        
    X_reg = df_reg.drop('AveragePrice', axis=1).values
    y_reg = df_reg['AveragePrice'].values
    
    X_train_reg, X_test_reg, y_train_reg, y_test_reg = train_test_split(X_reg, y_reg, test_size=0.2, random_state=42)
    
    scaler_reg = StandardScaler()
    X_train_reg = scaler_reg.fit_transform(X_train_reg)
    X_test_reg = scaler_reg.transform(X_test_reg)
    
    print("Regression Data Ready:", X_train_reg.shape)
except Exception as e:
    print("Error loading data:", e)

In [ ]:
# 1. Train Neural Network Regressor (Scratch)
input_dim_reg = X_train_reg.shape[1]
nn_reg = NeuralNetwork(layers_structure=[input_dim_reg, 64, 32, 1], learning_rate=0.001, task='regression')

print("Training Custom Neural Network Regressor...")
loss_hist_reg = nn_reg.fit(X_train_reg, y_train_reg, epochs=1000, verbose=False)

plt.plot(loss_hist_reg)
plt.title('Regression Training Loss (MSE)')
plt.show()

y_pred_nn_reg = nn_reg.predict(X_test_reg).flatten()
print(f"NN R2 Score: {r2_score(y_test_reg, y_pred_nn_reg):.4f}")
print(f"NN MAE: {mean_absolute_error(y_test_reg, y_pred_nn_reg):.4f}")

In [ ]:
# 2. Train Linear Regression (Scratch)
lin_reg = LinearRegressionScratch(learning_rate=0.01, iterations=2000)
print("Training Custom Linear Regression...")
lin_reg.fit(X_train_reg, y_train_reg)

y_pred_lin = lin_reg.predict(X_test_reg)
print(f"Linear Regression R2 Score: {r2_score(y_test_reg, y_pred_lin):.4f}")